In [ ]:
import pandas as pd

In [ ]:
pokemon_dt = pd.read_csv(r"DataBases\\df_pokemon.csv")
pokemon_dt=pokemon_dt.drop(columns=["Species",'Variant','Generation','Rarity','Evolves_from','Has_gender_diff','VGC2022_rules','Monthly Usage (k)','Usage Percent (%)','Monthly Rank'])

pokemon_dt.head(50)

In [ ]:
import pandas as pd
import numpy as np

type_chart=pd.read_csv(r"DataBases\\bridge_type_type_MOVE_EFFECTIVENESS_ON_POKEMON.csv")
type_chart["Damage Multiplier"].fillna(1,inplace=True)
type_chart['Damage Multiplier'] = type_chart['Damage Multiplier'].replace('½', '0.5').astype(float)
type_chart.head(25)
type_chart[(type_chart["Atk. Move Type"] == "Electric") & (type_chart["Def. Pokemon Type"] == "Flying")]

In [ ]:
import pandas as pd
import numpy as np

moves_ds =pd.read_csv(r"DataBases\df_moves.csv")
moves_ds=moves_ds.fillna(0.0,inplace=False)


In [ ]:
import pandas as pd
import numpy as np

moves_ds =pd.read_csv(r"DataBases\df_moves.csv")
moves_ds.rename(columns={"Prob. (%)":"Prob"},inplace=True)
moves_with_effect=moves_ds.where(moves_ds["Prob"].notna()).dropna()
status_moves=moves_ds.where(moves_ds["Damage_class"]=="Status").dropna(how="all")
df=moves_with_effect.where(moves_with_effect["Prob"]==100).dropna(how="all")
#df["Effect"].str.contains("Special Defense").any()
moves_ds.where(moves_ds["Name"]=="Flare Blitz").dropna(how="all")

In [ ]:
import pandas as pd
pokemove = pd.read_csv(r"DataBases\\bridge_pokemon_moves_MAY_LEARN.csv")
pokemoves_dict = pokemove.groupby('Pokemon')['Move'].apply(list).to_dict()

def pokemon_with_move(desired_move):
    pokemon_with_desired_move = [pokemon for pokemon, moves in pokemoves_dict.items() if desired_move in moves]
    print(f"Pokemon that can learn '{desired_move}': {pokemon_with_desired_move}, {len(pokemon_with_desired_move)} pokemons")

pokemon_with_move("Beak Blast")

In [2]:
import customtkinter as ctk
from tkinter.ttk import Progressbar
from PIL import Image, ImageOps
import pandas as pd
import requests
from io import BytesIO
from Pokemon import pokemon
import random
import socket
import pickle
import threading
import queue

pokemove = pd.read_csv(r"DataBases\\bridge_pokemon_moves_MAY_LEARN.csv")
pokemon_dt = pd.read_csv(r"DataBases\\df_pokemon.csv")
pokemon_dt = pokemon_dt.drop(columns=["Species", 'Variant', 'Generation', 'Rarity', 'Evolves_from', 'Has_gender_diff', 'VGC2022_rules', 'Monthly Usage (k)', 'Usage Percent (%)', 'Monthly Rank'])
# pokemon_dt=pokemon_dt.drop_duplicates(subset="ID",keep="first")
pokemon_dt = pokemon_dt.set_index("ID")

def recvall(sock):
    data = b''
    while True:
        part = sock.recv(4096)
        data += part
        if len(part) < 4096:
            # either 0 or end of data
            break
    return data


class PokemonFrame(ctk.CTkFrame):
    def __init__(self, master, pokemon, pokemon_hp, width=250, height=100, **kwargs):
        super().__init__(master, width=width, height=height, **kwargs)

        self.pokemon_name_label = ctk.CTkLabel(self, text=pokemon.Name)
        self.pokemon_name_label.pack()

        self.hp_label = ctk.CTkLabel(self, text=f"HP: {pokemon_hp}")
        self.hp_label.pack()

        self.hp_bar = Progressbar(self, length=width, mode='determinate', value=pokemon_hp, maximum=pokemon_hp)
        self.hp_bar.pack()

        cond_img = Image.open("Assets/BRN.png")
        self.cond_img = ctk.CTkImage(cond_img, size=(100, 50))
        self.cond_label = ctk.CTkLabel(self, image=self.cond_img, text="")

    def place_condition(self, img):
        self.cond_label.configure(image=ctk.CTkImage(img, size=(100, 50)))
        self.cond_label.pack()

    def clear_condition(self):
        self.cond_label.forget()

    def update_hp(self, new_hp,max_hp=None):
        if max_hp is not None:
            self.hp_bar.configure(maximum=max_hp)
        if new_hp >= 0:
            self.hp_bar.configure(value=new_hp)
            self.hp_label.configure(text=f"HP: {new_hp}")
        else:
            self.hp_bar.configure(value=0)
            self.hp_label.configure(text="HP: 0")


class SwitchOrMove(ctk.CTkFrame):
    TYPE_COLORS = {
        'Normal': '#A8A77A',
        'Fire': '#EE8130',
        'Water': '#6390F0',
        'Electric': '#F7D02C',
        'Grass': '#7AC74C',
        'Ice': '#96D9D6',
        'Fighting': '#C22E28',
        'Poison': '#A33EA1',
        'Ground': '#E2BF65',
        'Flying': '#A98FF3',
        'Psychic': '#F95587',
        'Bug': '#A6B91A',
        'Rock': '#B6A136',
        'Ghost': '#735797',
        'Dragon': '#6F35FC',
        'Dark': '#705746',
        'Steel': '#B7B7CE',
        'Fairy': '#D685AD',
    }

    def cache_team_images(self, team):
        for pokemon in team:
            if pokemon.HP >= 0:
                url = pokemon.img
                res = requests.get(url)
                img = Image.open(BytesIO(res.content))
                img.save(f"Cached_Images/{pokemon.Name}.png")

    def __init__(self, master, team_pokemon, selected_pokemon, opponent_pokemon, **kwargs):
        super().__init__(master, **kwargs)
        self.team = team_pokemon
        self.selected_pokemon = selected_pokemon
        self.opponent_pokemon = opponent_pokemon
        self.master = master
        self.cache_team_images(self.team)

        self.configure(width=375, height=300)
        self.attack_button = ctk.CTkButton(self, text="Attack", command=lambda: self.show_moveset(self.selected_pokemon.move_list, self.selected_pokemon.moveset))
        self.attack_button.place(relx=0, rely=0)

        self.switch_button = ctk.CTkButton(self, text="Switch", command=self.show_team)
        self.switch_button.place(relx=0, rely=0.15)

        self.button1 = ctk.CTkButton(self, text="none", command=lambda: self.perform_move(self.master, self.button1.cget("text")))
        self.button2 = ctk.CTkButton(self, text="none", command=lambda: self.perform_move(self.master, self.button2.cget("text")))
        self.button3 = ctk.CTkButton(self, text="none", command=lambda: self.perform_move(self.master, self.button3.cget("text")))
        self.button4 = ctk.CTkButton(self, text="none", command=lambda: self.perform_move(self.master, self.button4.cget("text")))

        self.pokebutton1 = ctk.CTkButton(self, width=100, height=100)
        self.pokebutton2 = ctk.CTkButton(self, width=100, height=100)
        self.pokebutton3 = ctk.CTkButton(self, width=100, height=100)
        self.pokebutton4 = ctk.CTkButton(self, width=100, height=100)
        self.pokebutton5 = ctk.CTkButton(self, width=100, height=100)
        self.pokebutton6 = ctk.CTkButton(self, width=100, height=100)

        self.infobox = ctk.CTkTextbox(self, width=325, height=100)

        self.wait_box=ctk.CTkTextbox(self,width=375,height=300,font=("Ariel",50))
        self.wait_box.configure(state="normal")
        self.wait_box.insert("0.0","Wait...")
        self.wait_box.configure(state="disabled")

    def return_def(self):
        self.switch_button.place(relx=0, rely=0.15)
        self.attack_button.place(relx=0, rely=0)

        self.return_button.place_forget()
        self.infobox.place_forget()
        for i in range(1, 5):  # Loop through the attack buttons
            b = getattr(self, f'button{i}')
            b.place_forget()  # Forget the button
        for i in range(1, 7):
            b = getattr(self, f'pokebutton{i}')
            b.place_forget()
        self.wait_box.pack_forget()

    def wait_for_turn(self):
        self.switch_button.place_forget()
        self.attack_button.place_forget()

        self.return_button.place_forget()
        self.infobox.place_forget()
        for i in range(1, 5):  # Loop through the attack buttons
            b = getattr(self, f'button{i}')
            b.place_forget()  # Forget the button
        for i in range(1, 7):
            b = getattr(self, f'pokebutton{i}')
            b.place_forget()
        self.wait_box.pack()

    def perform_move(self, master, move):
        def send_move_and_receive_result():
            client_socket.send(f"move:{move}".encode())
            self.wait_for_turn()
            result = client_socket.recv(1024).decode()
            self.return_def()

            client_socket.send("Give results".encode())

            pokemon_data = recvall(client_socket)
            self.selected_pokemon = pickle.loads(pokemon_data)
            pokemon_data = recvall(client_socket)
            self.opponent_pokemon = pickle.loads(pokemon_data)

            master.m_pokemon_frame.update_hp(self.selected_pokemon.HP)
            master.e_pokemon_frame.update_hp(self.opponent_pokemon.HP)

            master.result_box.configure(state="normal")
            master.result_box.delete('0.0', "end")
            master.result_box.insert("0.0", result)
            master.result_box.configure(state="disabled")

            if self.selected_pokemon.has_status_effect():
                img_dir = "BRN" if self.selected_pokemon.burn else "PAR" if self.selected_pokemon.paralyze else "PSN" if self.selected_pokemon.poison else "FRZ" if self.selected_pokemon.freeze else ""
                if img_dir != "":
                    img = Image.open(f"Assets/{img_dir}.png")
                    master.m_pokemon_frame.place_condition(img)
            else:
                master.m_pokemon_frame.clear_condition()

            if self.opponent_pokemon.has_status_effect():
                img_dir = "BRN" if self.opponent_pokemon.burn else "PAR" if self.opponent_pokemon.paralyze else "PSN" if self.opponent_pokemon.poison else "FRZ" if self.opponent_pokemon.freeze else ""
                if img_dir != "":
                    img = Image.open(f"Assets/{img_dir}.png")
                    master.e_pokemon_frame.place_condition(img)
            else:
                master.e_pokemon_frame.clear_condition()

            if self.selected_pokemon.HP <= 0:
                self.return_def()
                self.show_team(show_return=False)
            else:
                self.return_def()

        move_thread = threading.Thread(target=send_move_and_receive_result)
        move_thread.start()

    def show_moveset(self, moves, moveset):
        self.switch_button.place_forget()
        self.attack_button.place_forget()
        i = 0
        for m in moves:
            if i <= 4:  # Only update the buttons if there are 4 or fewer moves
                b = getattr(self, f'button{i+1}')
                b.configure(text=m, fg_color=self.TYPE_COLORS.get(moveset["Type"].iloc[i]))
                b.bind('<Enter>', lambda event, move=m: self.show_move_info(move, moveset))
                b.bind('<Leave>', lambda event: self.clear_move_info())
                b.place(relx=0, rely=0.15 * i)
                i += 1

        self.return_button = ctk.CTkButton(self, text="return", command=self.return_def)
        self.return_button.place(relx=0.5, rely=0)

        self.infobox.place(rely=0.6, relx=0.05)
        self.infobox.configure(state="disabled")

    def show_move_info(self, move, moveset):
        move_info = moveset.loc[moveset['Name'] == move]
        info_text = f"Name: {move_info['Name'].iloc[0]}\nType: {move_info['Type'].iloc[0]}\nPower: {move_info['Power'].iloc[0]}\nEffect: {move_info['Effect'].iloc[0]}"
        self.infobox.configure(state="normal")
        self.infobox.delete('1.0', 'end')
        self.infobox.insert('1.0', info_text)
        self.infobox.configure(state="disabled")

    def clear_move_info(self):
        self.infobox.configure(state="normal")
        self.infobox.delete('1.0', 'end')
        self.infobox.configure(state="disabled")

    def show_team(self, show_return=True):
        team = self.team
        self.switch_button.place_forget()
        self.attack_button.place_forget()
        i = 0
        counter = 0
        for t in team:
            if t.HP >= 0 and t.Name!=self.selected_pokemon.Name:
                img = Image.open(f"Cached_Images/{t.Name}.png")
                img = ctk.CTkImage(img, size=(100, 100))

                setattr(self, f'pokebutton{i + 1}', ctk.CTkButton(self, width=100, height=100, text="", image=img, command=lambda p=t: self.switch_pokemon(p)))
                b = getattr(self, f'pokebutton{i + 1}')
                b.configure(image=img, text="")
                if counter <= 2:
                    b.place(relx=counter * 0.33, rely=0)
                else:
                    b.place(relx=(counter - 3) * 0.33, rely=0.5)
                i += 1
                counter += 1
        if show_return:
            self.return_button = ctk.CTkButton(self, text="return", command=self.return_def)
            self.return_button.place(relx=0.25, rely=0.9)

    def switch_pokemon(self, new_pokemon):
        '''self.selected_pokemon = new_pokemon
        self.master.m_pokemon_frame.pokemon_name_label.configure(text=new_pokemon.Name)
        self.master.m_pokemon_frame.update_hp(new_pokemon.HP,max_hp=new_pokemon.max_HP)

        m_url = new_pokemon.img
        m_response = requests.get(m_url)
        m_img = Image.open(BytesIO(m_response.content))
        m_img = ImageOps.mirror(m_img)
        m_pokemon_img = ctk.CTkImage(m_img, size=(250, 250))
        self.master.m_pokemon_label.configure(image=m_pokemon_img)
        self.return_def()

        self.master.m_pokemon_frame.clear_condition()

        if new_pokemon.has_status_effect():
            img_dir = "BRN" if new_pokemon.burn else "PAR" if new_pokemon.paralyze else "PSN" if new_pokemon.poison else "FRZ" if new_pokemon.freeze else ""
            if img_dir != "":
                img = Image.open(f"Assets/{img_dir}.png")
                self.master.m_pokemon_frame.place_condition(img)'''
        def send_switch_receive_result():
            client_socket.send(f"switch:{new_pokemon.Name}".encode())
            self.wait_for_turn()
            result = client_socket.recv(1024).decode()
            self.return_def()
        
        switch_thread = threading.Thread(target=send_switch_receive_result)
        switch_thread.start()
        



class Game_Screen(ctk.CTkFrame):
    def __init__(self, master,team,selected_pokemon,opponent_pokemon, **kwargs):
        super().__init__(master, **kwargs)
        self.configure(width=1920, height=1080)

        m_url = selected_pokemon.img
        m_response = requests.get(m_url)
        m_img = Image.open(BytesIO(m_response.content))
        m_img = ImageOps.mirror(m_img)
        m_pokemon_img = ctk.CTkImage(m_img, size=(250, 250))
        self.m_pokemon_label = ctk.CTkLabel(master=self, image=m_pokemon_img, text="")
        self.m_pokemon_label.place(relx=0.3, rely=0.5)

        e_url = opponent_pokemon.img
        e_response = requests.get(e_url)
        e_img = Image.open(BytesIO(e_response.content))
        e_pokemon_img = ctk.CTkImage(e_img, size=(250, 250))
        e_pokemon_label = ctk.CTkLabel(master=self, image=e_pokemon_img, text="")
        e_pokemon_label.place(relx=0.55, rely=0)

        vs_img = Image.open(r"Assets/vs_img.png")
        vs_img = ctk.CTkImage(vs_img, size=(1200, 200))
        vs_label = ctk.CTkLabel(self, image=vs_img, text="")
        vs_label.place(rely=0.3, relx=0.18)

        self.result_box = ctk.CTkTextbox(self, width=1500, height=200)
        self.result_box.insert('0.0', "THIS IS A RESULTS TEXT BOX")
        self.result_box.configure(state="disabled")
        self.result_box.place(relx=0.1, rely=0.8)

        self.m_pokemon_frame = PokemonFrame(self, selected_pokemon, selected_pokemon.HP, width=250, height=150)
        self.m_pokemon_frame.place(relx=0.45, rely=0.6)

        self.switchormove = SwitchOrMove(self,team, selected_pokemon, opponent_pokemon)
        self.switchormove.place(relx=0.6, rely=0.5)

        self.e_pokemon_frame = PokemonFrame(self, opponent_pokemon, opponent_pokemon.HP,width=250, height=150)
        self.e_pokemon_frame.place(relx=0.4, rely=0.1)


# setting root
ctk.set_appearance_mode("dark")
root = ctk.CTk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

res = f"{screen_width}x{screen_height}"
root.title("PokeBattle")
root.geometry("1920x1080")
# root.attributes("-fullscreen", "True")
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

team = [pokemon(pokemon_dt.loc[pokemon_dt['Name'] == 'Toucannon']),
                   pokemon(pokemon_dt.loc[pokemon_dt['Name'] == 'Charizard']),
                   pokemon(pokemon_dt.loc[pokemon_dt['Name'] == 'Pikachu']),
                   pokemon(pokemon_dt.loc[pokemon_dt['Name'] == 'Alakazam']),
                   pokemon(pokemon_dt.loc[pokemon_dt['Name'] == 'Hariyama']),
                   pokemon(pokemon_dt.loc[pokemon_dt['Name'] == 'Dragonite'])
                   ]




start=False

# Create a socket object
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostname()
port = 12345
client_socket.connect((host, port))


team_data = pickle.dumps(team)
client_socket.sendall(team_data)

# Wait for the battle to start
print("Waiting for the battle to start...")
while not start:
    data=client_socket.recv(1024).decode()
    if data=="start":start=True
print("battle starts!!")


pokemon_data = recvall(client_socket)
selected_pokemon = pickle.loads(pokemon_data)
pokemon_data = recvall(client_socket)
opponent_pokemon = pickle.loads(pokemon_data)

game = Game_Screen(root,team,selected_pokemon,opponent_pokemon)
game.pack()

root.mainloop()


Waiting for the battle to start...
battle starts!!


KeyboardInterrupt: 

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.7_3.7.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.7_3.7.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Itay\AppData\Local\Temp\ipykernel_18968\3418091800.py", line 163, in send_move_and_receive_result
    self.return_def()
  File "C:\Users\Itay\AppData\Local\Temp\ipykernel_18968\3418091800.py", line 131, in return_def
    self.switch_button.place(relx=0, rely=0.15)
  File "C:\Users\Itay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py", line 274, in place
    return super().place(**self._apply_argument_scaling(kwargs))
  F

In [ ]:
import pandas as pd
import re

# Read your dataset
pokemove = pd.read_csv(r"DataBases\\bridge_pokemon_moves_MAY_LEARN.csv")
moves_ds = pd.read_csv(r"DataBases\df_moves.csv")
moves_ds.rename(columns={"Prob. (%)": "Prob"}, inplace=True)
moves_ds = moves_ds.dropna(subset=["Damage_class"])
moves_ds["Effect"] = moves_ds["Effect"].astype(str).str.lower()
moves_ds= moves_ds[~((moves_ds["Damage_class"].isin(["Special", "Physical"])) & (moves_ds["Power"].isna()))]

# Define regular expression patterns for different effect types
lower_stat_pattern = r"(?i)lowers? ?\w* (opponent's|user's|target's) (\w+)"
raise_stat_pattern = r"(?i)raises? ?\w* (opponent's|user's) (\w+)"
special_stat_pattern = r"(?i)lowers? ?\w* (opponent's|user's|target's) special (\w+)|raises? ?\w* (opponent's|user's|target's) special (\w+)"
hit_multiple= r"(?i)Hits (\d)-(\d) times"
status_condition= r"(?i)(poison|paralyze|burn|freeze|confuse)"
traps = r"(?i)traps? opponent"
recoil = r"(?i)receives? recoil damage"
atk_first=r"(?i)(attacks|goes) first"
high_crit=r"(?i)high critical hit ratio"
flinch=r"(?i)cause flinch"
drain=r"(?i)hp inflicted"
recover=r'(?i)recovers half its max hp'

# Define a function to translate move effects using regular expressions
def translate_effect(effect):
    lower_match = re.search(lower_stat_pattern, effect)
    raise_match = re.search(raise_stat_pattern, effect)
    special_match = re.search(special_stat_pattern, effect)
    hit_multiple_match = re.search(hit_multiple, effect)
    status_condition_match = re.search(status_condition, effect)
    traps_match = re.search(traps, effect)
    recoil_match = re.search(recoil, effect)
    atk_first_match = re.search(atk_first, effect)
    high_crit_match=re.search(high_crit,effect)
    flinch_match=re.search(flinch,effect)
    drain_match=re.search(drain,effect)
    recover_match=re.search(recover,effect)
    move_effect=""
    effects = []


    if special_match:
        whom = special_match.group(1) or special_match.group(3)
        #print(type(special_match.group(4)))
        stat = f"special {special_match.group(4)}" if special_match.group(2)==None else f"special {special_match.group(2)}"
        effect_type = "lower" if special_match.group(1) else "raise"
        whom = whom.replace("target", "opponent")
        effects.append(f"stat change: ({whom[:-2]},{effect_type},{stat})")
    elif raise_match:
        whom = raise_match.group(1)
        stat = raise_match.group(2)
        effect_type = "raise"
        whom = whom.replace("target", "opponent")
        effects.append(f"stat change: ({whom[:-2]},{effect_type},{stat})")
    elif lower_match:
        whom = lower_match.group(1)
        stat = lower_match.group(2)
        effect_type = "lower"
        whom = whom.replace("target", "opponent")
        effects.append(f"stat change: ({whom[:-2]},{effect_type},{stat})")

    if hit_multiple_match:
        effects.append("multihit")
    if status_condition_match:
        effects.append(status_condition_match.group(1))
    if traps_match:
        effects.append("traps")
    if recoil_match:
        effects.append("recoil")
    if atk_first_match:
        effects.append("first")
    if high_crit_match:
        effects.append("critical")
    if flinch_match:
        effects.append("flinch")
    if drain_match:
        effects.append("drain")
    if recover_match:
        effects.append("recover")

    # Combine all effects into a comma-separated string
    move_effect += ",".join(effects)
    if move_effect=="":
        move_effect="nan"

    return move_effect

# Apply the translation function to create the Move_effect column
moves_ds["Move_effect"] = moves_ds["Effect"].apply(translate_effect)
moves_ds = moves_ds.where((moves_ds["Move_effect"] != "nan") | (moves_ds["Damage_class"] != "Status")).dropna(how="all")

chosed_moves= pokemove.where(pokemove['Pokemon']=="Mew").dropna(how='all')
moveset = chosed_moves.merge(moves_ds,left_on="Move",right_on="Name")
moveset=moveset.drop(columns=['Pokemon','Move','Introducted_in'])
moves_ds.where(moves_ds["Move_effect"]=="burn").dropna(how="all")
